1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)
2. сделать feature engineering
3. обучить любой классификатор (какой вам нравится)
4. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
5. применить random negative sampling для построения классификатора в новых условиях
6. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)
7. поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

Выбранный датасет https://archive.ics.uci.edu/ml/datasets/Online+Shoppers+Purchasing+Intention+Dataset

Набор данных состоит из 10 числовых и 8 категориальных признаков.
Признак "Revenue" можно использовать в качестве метки класса.

In [61]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [62]:
import pandas as pd
import numpy as np

In [163]:
#загрузим dataset
df = pd.read_csv('online_shoppers_intention.csv')
df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False


In [164]:
df.shape

(12330, 18)

In [165]:
len(df) - df.count()

Administrative             0
Administrative_Duration    0
Informational              0
Informational_Duration     0
ProductRelated             0
ProductRelated_Duration    0
BounceRates                0
ExitRates                  0
PageValues                 0
SpecialDay                 0
Month                      0
OperatingSystems           0
Browser                    0
Region                     0
TrafficType                0
VisitorType                0
Weekend                    0
Revenue                    0
dtype: int64

In [166]:
df['Revenue'] = df['Revenue'].astype(int)

In [167]:
df['Weekend'] = df['Weekend'].astype(int)

In [168]:
df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,0,0
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,0,0
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,0,0
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,0,0
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,1,0


In [169]:
df['OperatingSystems'].value_counts()

2    6601
1    2585
3    2555
4     478
8      79
6      19
7       7
5       6
Name: OperatingSystems, dtype: int64

In [170]:
df['Administrative'].value_counts()

0     5768
1     1354
2     1114
3      915
4      765
5      575
6      432
7      338
8      287
9      225
10     153
11     105
12      86
13      56
14      44
15      38
16      24
17      16
18      12
19       6
22       4
24       4
23       3
21       2
20       2
27       1
26       1
Name: Administrative, dtype: int64

In [171]:
df['Informational'].value_counts()

0     9699
1     1041
2      728
3      380
4      222
5       99
6       78
7       36
9       15
8       14
10       7
12       5
14       2
24       1
16       1
11       1
13       1
Name: Informational, dtype: int64

In [172]:
df['ProductRelated'].value_counts()

1      622
2      465
3      458
4      404
6      396
      ... 
282      1
290      1
330      1
338      1
339      1
Name: ProductRelated, Length: 311, dtype: int64

In [173]:
df['SpecialDay'].value_counts()

0.0    11079
0.6      351
0.8      325
0.4      243
0.2      178
1.0      154
Name: SpecialDay, dtype: int64

In [174]:
df['PageValues'].value_counts()

0.000000     9600
53.988000       6
42.293068       3
16.158558       2
12.558857       2
             ... 
48.453707       1
8.052496        1
19.714689       1
1.104440        1
15.756363       1
Name: PageValues, Length: 2704, dtype: int64

In [175]:
df['Browser'].value_counts()

2     7961
1     2462
4      736
5      467
6      174
10     163
8      135
3      105
13      61
7       49
12      10
11       6
9        1
Name: Browser, dtype: int64

In [176]:
df['Region'].value_counts()

1    4780
3    2403
4    1182
2    1136
6     805
7     761
9     511
8     434
5     318
Name: Region, dtype: int64

In [177]:
df['TrafficType'].value_counts()

2     3913
1     2451
3     2052
4     1069
13     738
10     450
6      444
8      343
5      260
11     247
20     198
9       42
7       40
15      38
19      17
14      13
18      10
16       3
12       1
17       1
Name: TrafficType, dtype: int64

In [178]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


from sklearn.preprocessing import StandardScaler


continuos_cols = ['Administrative','Administrative_Duration', 'Informational', 'Informational_Duration', 
                  'ProductRelated', 'ProductRelated_Duration', 'BounceRates', 'ExitRates', 'PageValues', 'TrafficType']
cat_cols = ['SpecialDay', 'Month', 'OperatingSystems', 'Browser', 'Region', 'VisitorType']
base_cols = ['Weekend']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

In [179]:
#объединим трансформеры с помощью featureunion
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers + cat_transformers + base_transformers)
feature_processing = Pipeline([('feats', feats)])



In [180]:
X = df.drop('Revenue', axis=1)
y = df['Revenue']
data = feature_processing.fit_transform(X)

In [181]:
feature_processing.fit_transform(X).shape

(12330, 60)

In [182]:
#разделим данные на train/test
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.3, random_state=24)

In [183]:
X_train.shape

(8631, 60)

In [184]:
X_test.shape

(3699, 60)

In [185]:
#обучаем модель
import xgboost as xgb

model = xgb.XGBClassifier()

model.fit(X_train, y_train)
y_predict = model.predict(X_test)

[00:37:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [186]:
#проверяем качество
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 

    
evaluate_results(y_test, y_predict)

Classification results:
f1: 62.06%
roc: 76.27%
recall: 57.20%
precision: 67.83%


In [187]:
#представим, что нам неизвестны позитивы и часть негативов
mod_data = pd.DataFrame(data.copy())
mod_y = y.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_y.values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 477/1908 as positives and unlabeling the rest


In [188]:
#Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)
mod_data['class_test'] = -1
mod_data.loc[pos_sample, 'class_test'] = 1
print('target variable:\n', mod_data['class_test'].value_counts())

target variable:
 -1    11853
 1      477
Name: class_test, dtype: int64


In [189]:
mod_data.head(10)

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,class_test
0,-0.696993,-0.457191,-0.396478,-0.244931,-0.691003,-0.624348,3.667189,3.229316,-0.317178,-0.762629,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1
1,-0.696993,-0.457191,-0.396478,-0.244931,-0.668518,-0.590903,-0.457683,1.171473,-0.317178,-0.514182,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1
2,-0.696993,-0.457191,-0.396478,-0.244931,-0.691003,-0.624348,3.667189,3.229316,-0.317178,-0.265735,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,-1
3,-0.696993,-0.457191,-0.396478,-0.244931,-0.668518,-0.622954,0.573535,1.994610,-0.317178,-0.017289,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1
4,-0.696993,-0.457191,-0.396478,-0.244931,-0.488636,-0.296430,-0.045196,0.142551,-0.317178,-0.017289,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-1
5,-0.696993,-0.457191,-0.396478,-0.244931,-0.286270,-0.543757,-0.132035,-0.380935,-0.317178,-0.265735,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1
6,-0.696993,-0.457191,-0.396478,-0.244931,-0.691003,-0.624348,3.667189,3.229316,-0.317178,-0.265735,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1
7,-0.395938,-0.457191,-0.396478,-0.244931,-0.713488,-0.624348,3.667189,3.229316,-0.317178,0.231158,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-1
8,-0.696993,-0.457191,-0.396478,-0.244931,-0.668518,-0.605012,-0.457683,1.171473,-0.317178,-0.265735,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1
9,-0.696993,-0.457191,-0.396478,-0.244931,-0.646033,-0.238685,-0.457683,-0.429072,-0.317178,-0.514182,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1


In [190]:
x_data = mod_data.iloc[:,:-1].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_y # original class

### random negative sampling

In [191]:
mod_data['y'] = mod_y

In [192]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(477, 62) (477, 62)


In [193]:
model = xgb.XGBClassifier()

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-1].values)
y_predict_rns = model.predict(sample_test.iloc[:,:-2].values)
evaluate_results(sample_test.iloc[:,-1].values, y_predict_rns)

[00:37:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Classification results:
f1: 54.84%
roc: 83.50%
recall: 83.85%
precision: 40.75%


In [194]:
evaluate_results(y_test, y_predict)

Classification results:
f1: 62.06%
roc: 76.27%
recall: 57.20%
precision: 67.83%


In [ ]:
#качество выросло на roc-auc и recall

### если 35%

In [195]:
#представим, что нам неизвестны позитивы и часть негативов
mod_data_35 = pd.DataFrame(data.copy())
mod_y_35 = y.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_y_35.values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 35% of the positives marked
pos_sample_len_35 = int(np.ceil(0.35 * len(pos_ind)))
print(f'Using {pos_sample_len_35}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample_35 = pos_ind[:pos_sample_len_35]

Using 668/1908 as positives and unlabeling the rest


In [196]:
#Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)
mod_data_35['class_test'] = -1
mod_data_35.loc[pos_sample_35, 'class_test'] = 1
print('target variable:\n', mod_data_35['class_test'].value_counts())

target variable:
 -1    11662
 1      668
Name: class_test, dtype: int64


In [197]:
mod_data_35['y'] = mod_y

In [198]:
mod_data_35 = mod_data_35.sample(frac=1)
neg_sample = mod_data_35[mod_data_35['class_test']==-1][:len(mod_data_35[mod_data_35['class_test']==1])]
sample_test = mod_data_35[mod_data_35['class_test']==-1][len(mod_data_35[mod_data_35['class_test']==1]):]
pos_sample = mod_data_35[mod_data_35['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(668, 62) (668, 62)


In [199]:
model = xgb.XGBClassifier()

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-1].values)
y_predict_rns = model.predict(sample_test.iloc[:,:-2].values)
evaluate_results(sample_test.iloc[:,-1].values, y_predict_rns)

[00:48:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Classification results:
f1: 51.07%
roc: 83.50%
recall: 84.50%
precision: 36.59%


In [ ]:
# подрос только recall при 35%

### если 15%

In [200]:
#представим, что нам неизвестны позитивы и часть негативов
mod_data_15 = pd.DataFrame(data.copy())
mod_y_15 = y.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_y_15.values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 15% of the positives marked
pos_sample_len_15 = int(np.ceil(0.15 * len(pos_ind)))
print(f'Using {pos_sample_len_15}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample_15 = pos_ind[:pos_sample_len_15]

Using 287/1908 as positives and unlabeling the rest


In [201]:
#Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)
mod_data_15['class_test'] = -1
mod_data_15.loc[pos_sample_15, 'class_test'] = 1
print('target variable:\n', mod_data_15['class_test'].value_counts())

target variable:
 -1    12043
 1      287
Name: class_test, dtype: int64


In [202]:
mod_data_15['y'] = mod_y

In [203]:
mod_data_15 = mod_data_15.sample(frac=1)
neg_sample = mod_data_15[mod_data_15['class_test']==-1][:len(mod_data_15[mod_data_15['class_test']==1])]
sample_test = mod_data_15[mod_data_15['class_test']==-1][len(mod_data_15[mod_data_15['class_test']==1]):]
pos_sample = mod_data_15[mod_data_15['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(287, 62) (287, 62)


In [204]:
model = xgb.XGBClassifier()

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-1].values)
y_predict_rns = model.predict(sample_test.iloc[:,:-2].values)
evaluate_results(sample_test.iloc[:,-1].values, y_predict_rns)

[00:52:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Classification results:
f1: 55.11%
roc: 83.32%
recall: 86.60%
precision: 40.41%


In [ ]:
# recall самый высокий при 15%